In [ ]:
import pandas as pd
from pypfopt import EfficientFrontier, expected_returns, plotting, risk_models

from app.backtesting import Backtesting
from app.controller.crypto_controller import get_top_coins, get_coins_prices, get_coins_market_caps, get_coins_total_volumes

%matplotlib inline

In [ ]:
max_days: int = 180
min_days: int = 90
num_coins: int = 20

top_coingecko_coins = get_top_coins(num_coins)

prices = get_coins_prices(top_coingecko_coins, max_days, min_days)
market_caps = get_coins_market_caps(top_coingecko_coins, max_days, min_days)
total_volumes = get_coins_total_volumes(top_coingecko_coins, max_days, min_days)

In [ ]:
df_prices = pd.DataFrame.from_dict(prices, orient="index").sort_index().dropna()
df_prices.index = pd.to_datetime(df_prices.index)
df_prices.tail()

In [ ]:
df_mcaps = pd.DataFrame.from_dict(market_caps, orient="index").sort_index().dropna()
df_mcaps.index = pd.to_datetime(df_mcaps.index)
df_mcaps.tail()

In [ ]:
df_vols = pd.DataFrame.from_dict(total_volumes, orient="index").sort_index().dropna()
df_vols.index = pd.to_datetime(df_vols.index)
df_vols.tail()

In [ ]:
mu = expected_returns.capm_return(df_prices, frequency=365)

In [ ]:
S = risk_models.CovarianceShrinkage(df_prices, frequency=365).oracle_approximating()

In [ ]:
print("Correlation")
correlation_plot = plotting.plot_covariance(S, plot_correlation=True, dpi=500)
correlation_plot.figure.set_size_inches((12, 12))

In [ ]:
print("Covariance")
covariance_plot = plotting.plot_covariance(S, dpi=500)
covariance_plot.figure.set_size_inches((12, 12))

In [ ]:
backtest = Backtesting(df_prices, add_raw_tickers=True)

In [ ]:
ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights(rounding=2)
backtest.add_strategy(weights, name="ef_min_vol")
ef.portfolio_performance(verbose=True)

In [ ]:
for asset, weight in sorted(weights.items(), key=lambda x: x[1], reverse=True):
    if weight > 0:
        print(f"{asset}: {weight}")


In [ ]:
ef = EfficientFrontier(mu, S)
ef.max_sharpe()
weights = ef.clean_weights(rounding=2)
backtest.add_strategy(weights, name="ef_max_sharpe")
ef.portfolio_performance(verbose=True)

In [ ]:
for asset, weight in sorted(weights.items(), key=lambda x: x[1], reverse=True):
    if weight > 0:
        print(f"{asset}: {weight}")


In [ ]:
ef = EfficientFrontier(mu, S)
ef.max_quadratic_utility()
weights = ef.clean_weights(rounding=2)
backtest.add_strategy(weights, name="ef_max_quadratic")
ef.portfolio_performance(verbose=True)

In [ ]:
for asset, weight in sorted(weights.items(), key=lambda x: x[1], reverse=True):
    if weight > 0:
        print(f"{asset}: {weight}")

In [ ]:
results = backtest.run()
results.display()


In [ ]:
results.plot(freq="D", figsize=(20, 10), title="Capital gains")